In [2]:
!python --version

Python 3.9.13


# Do not run subsequent cell unless t5-large.pt doesn't exist

In [6]:
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
#import torch    

#model = AutoModelForSeq2SeqLM.from_pretrained("t5-large") #, revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True)
#tokenizer = AutoTokenizer.from_pretrained("t5-large")

#torch.save(model, "t5-large.pt")
#torch.save(tokenizer, "tokenizer-t5-large.pt")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# Load Presaved Model

In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = torch.load("t5-large.pt")

tokenizer = torch.load("tokenizer-t5-large.pt")

In [7]:
from transformers import pipeline
# create pipeline
gen = pipeline("summarization",model=model,tokenizer=tokenizer,device=0)

text_data = """
When the summer sun is blazing, the mere idea of turning on the oven — even for the most wonderful dish — is revolting. Sure, you may have air conditioning, but why heat up the kitchen if you don’t have to? (You came inside to escape the stickiness and sweat!) You could artfully compose a salad or assemble a no-cook meal — and we have so many lovely ones here and here — but eventually, you’re going to want a warm meal. Enter the slow cooker.

The recipes below are the easiest kind of slow cooker recipe, the fix-it-and-forget-it sort. There is no presearing or sautéing, and very little prep. You just toss the ingredients into the crock and let it simmer away while you head out to the beach or to work, or just curl up with a good book in front of the air-conditioner. Several hours later, dinner is ready.
1. Slow Cooker Kofte in Tomato-Lime Broth
Image
Two kofte sit atop a dark bowl strewn with herbs and cooked-down tomatoes. At the bottom of the bowl is orzo in broth.
Credit...Joel Goldberg for The New York Times. Food Stylist: Barrett Washburne.
Two kofte sit atop a dark bowl strewn with herbs and cooked-down tomatoes. At the bottom of the bowl is orzo in broth.

Ali Slagle offers the perfect summer comfort meal with this dish of spiced meatballs simmered with cherry tomatoes and lime. Prepared in the style of kofte, the ground meat is combined with fresh herbs, turmeric and yogurt, which helps bind the meat and provides a lovely tang. Feel free to use lamb, beef, turkey or a combination. (One reader used alternative meat with happy results.) Serve over a pile of buttered orzo and sprinkle with fresh herbs.

"""

# run prediction
gen(text_data)


[{'summary_text': 'the recipes below are the easiest kind of slow cooker recipe, the fix-it-and-forget-it sort . there is no presearing or sautéing, and very little prep . Ali slagle offers the perfect summer comfort meal with spiced meatballs .'}]